In [1]:
import pandas as pd
import numpy as np
import re
import lxml
import seaborn as sns
import json

from bs4 import BeautifulSoup
from requests import get
%matplotlib inline

# This section will scrape IMDB and Box Office Mojo

In [2]:
#this functions creates the list of URLS based on size of films entered
def createURLs(numMovies): 
    """Create and return two lists of URLS given the number of entries desired
        Keyword Arguments:
        numMovies- the number of movies you want to scrape
    """
    
    
    
    url_range_imdb = range(1,numMovies,50) #create imdb range
    url_range_boxoffice = range(1,numMovies,200) #create box office mojo range
    url_imdb = []
    url_boxoffice = []
    for i in url_range_imdb: #create list of imdb urls
        url_imdb.append("https://www.imdb.com/search/title/?title_type=feature&start=" + str(i) + "&ref_=adv_prv")
    for i in url_range_boxoffice: #create list of box offic emojo urls
        if i == 1:
            url_boxoffice.append("https://www.boxofficemojo.com/chart/top_lifetime_gross/")
        else:
            url_boxoffice.append("https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=" + str(i-1))
    #creates a list of the urls to scrape from. Each page shows 50 results and starts from 1
    return (url_imdb, url_boxoffice) # return URLS




In [3]:
#initialize lists
title = []
date = []
genre = []
rating = []
score = []
director = []
stars = []
#loop through each url
url_list = createURLs(10000)
for url in url_list[0]:
    #grab the data from the url, read it into Beauitufl soup, find each movie from the page
    page = get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    content = soup.find(id="main") #grab the main content
    frames = content.find_all("div", class_="lister-item mode-advanced") #grab each movie intsance
    for frame in frames:    
        #grab data from each movie instance
        firstLine = frame.find("h3", class_="lister-item-header")
        title.append(firstLine.find("a").text)
        date.append(re.sub(r"[()]","", firstLine.find_all("span")[-1].text))
        try:
            genre.append(frame.find("span", class_="genre").text.rstrip().replace("\n","").split(","))
        except:
            genre.append('None')
        try:
            rating.append(frame.find("strong").text)
        except:
            rating.append(0.0) #set rating to 0.0 if its not there
        try:
            score.append(frame.find("span", class_="metascore favorable").text.rstrip())
        except:
            try:
                score.append(frame.find("span", class_="metascore unfavorable").text.rstrip())
            except:
                try:
                    score.append(frame.find("span", class_="metascore mixed").text.rstrip())
                except:
                    score.append(0.0) #set score to 0.0 if it doesnt exist
        cast = (frame.find("p", class_=""))
        try:    #split apart the director and the casts
            casts = cast.text.replace("\n","").split('|')
            casts = [x.strip() for x in casts]
            casts = [casts[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
            director.append(casts[0])
            stars.append([x.strip() for x in casts[1].split(",")])
        except:    #set director to 'No Director' if it isnt there
            casts = cast.text.replace("\n","").strip()
            director.append("No Director")
            stars.append([x.strip() for x in casts.split(",")])

    




In [4]:
#Creating the dataframe
column_names = ['Title','Year','Genre','Rating','Score','Director','Stars'] 
df = pd.DataFrame(list(zip(title,date,genre,rating,score,director,stars)),columns = column_names)





In [5]:

#Scrape box office mojo
titles = []
gross = []
years = []
for url in url_list[1]:
    page = get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    rows = soup.findAll('tr')
    i=0
    for row in rows:
        if i == 0:
            i=1
        else:
            titles.append(row.find(class_='a-link-normal').text)
            gross.append(row.find(class_='a-text-right mojo-field-type-money').text)
            years.append(row.find(class_='a-text-left mojo-field-type-year').text)



In [6]:
#Turn it into a dataframe
df2 = pd.DataFrame(list(zip(titles,gross,years)),columns = ['Title','Gross','Year'])


In [7]:
#merge the dataframes
merged_df= pd.merge(left = df, right = df2, on = ['Title','Year'])

In [8]:
merged_df.shape
merged_df.head()


,Title,Year,Genre,Rating,Score,Director,Stars,Gross
0,Parasite,2019,"[Comedy, Drama, Thriller]",8.6,96,Bong Joon Ho,"[Kang-ho Song, Sun-kyun Lee, Yeo-jeong Jo, Woo...","$51,809,136"
1,The Invisible Man,2020,"[Horror, Mystery, Sci-Fi]",7.6,71,Leigh Whannell,"[Elisabeth Moss, Oliver Jackson-Cohen, Harriet...","$33,738,175"
2,Knives Out,2019,"[Comedy, Crime, Drama]",8.0,82,Rian Johnson,"[Daniel Craig, Chris Evans, Ana de Armas, Jami...","$164,698,675"
3,Sonic the Hedgehog,2020,"[Action, Adventure, Comedy]",6.8,47,Jeff Fowler,"[Ben Schwartz, James Marsden, Jim Carrey, Tika...","$131,060,702"
4,Jojo Rabbit,2019,"[Comedy, Drama, War]",8.0,58,Taika Waititi,"[Roman Griffin Davis, Thomasin McKenzie, Scarl...","$33,176,865"


In [9]:
df.shape
#Clean the Gross and Genre columns
df['Gross'] = df['Gross'].astype(str)
df['Gross'] = df['Gross'].apply(lambda x: x.replace(',',''))
df['Gross'] = df['Gross'].apply(lambda x: x.replace('$',''))
df['Gross'] = df['Gross'].astype(int)
df['Genre'] = df['Genre'].apply(lambda x:x.strip('['))
df['Genre'] = df['Genre'].apply(lambda x:x.strip(']'))
df['Genre'] = df['Genre'].apply(lambda x:x.replace("'",''))
df['Genre'] = df['Genre'].apply(lambda x:x.replace(",",''))
df['Genre'] = df['Genre'].apply(lambda x:x.replace("  ",' '))



KeyError: 'Gross'

In [10]:
#output to csv
df.to_csv('movieData.csv', index = False)

# This section uses an API to get data from the Movie DB

In [ ]:
#retrieve theMovieDB.org API key
def get_keys(fname):
    '''
    Init signature: 
        get_keys(fname)
    Docstring:     
        get_keys(filename='') -> dict
        Retrieve API key stored on file.  return a dictionary.
    
    Parameters
    ----------
    fname: 
        json file name where the API key is stored.
      
    '''
    with open(fname) as f:
        return json.load(f)

In [ ]:

# Import the api key for movieDB using the get_keys function
keys = get_keys('moviedb.json')

# Assign API value to variable
api_key = keys['api_key']


Function get_mtdb_movies explainations:

The Movie DB web site provides an API to pull data from their source. However, they have a retriction of supplying on 10,000 movies or 500 pages of movies titles.
In order to collect all the movies, a for loop to issue the request for the movie data is a sound method.
The function defines the neccessary request parameters based on the API requirements.
In the for loop structure, we generate the respond obj, test the request status and proceed to create the dataframe on the first go. Subsequence passes, the new dataframe will be concatenated to the previous one. 

In [ ]:
# Use TMDB API to retrieve list of movies with primary_release_date greater than 01-01-2010
def get_mtdb_movies():
    '''
    init signature:
        get_mtdb_movies() 
    docstring:
        get_mtdb_movies() -> dataframe
        Using TMDB API to retrive a list of movies
    '''
    # Define get request parameters
    url = 'https://api.themoviedb.org/3/discover/movie?'
    langauge = 'en-US'
    sort_by = 'popularity.desc'
    include_adult = 'false'
    include_video = 'false'
    primary_release_date_gt = '2010-01-01'

    url_param = {
                    'api_key': api_key,
                    'language': langauge,
                    'sort_by': sort_by,
                    'include_adult': include_adult,
                    'include_video': include_video,
                    'primary_release_date.gt': primary_release_date_gt
                }

    for i in range(1,500): 
        # update the page number using the iteration value and pass it to the request parameter
        url_param.update({'page': i})
        # create the response ojb.
        resp = requests.get(url, params=url_param)
        # condition to test if this is the first page
        if resp.status_code == 200 and resp.json()['page'] == 1:
            # create the DataFrame
            df = pd.DataFrame.from_dict(resp.json()['results'])
        # condition to test it is not the first page
        elif resp.status_code == 200 and resp.json().get('page', 10000) < 10000:
            # convert response to dataframe
            a = pd.DataFrame.from_dict(resp.json()['results'])
            # concat the present df (a) with the previous pass.  Note: without the sort parameter, concat fails.
            df = pd.concat([df, a], sort=False)
        # condition to test end of the page
        elif resp.status_code == 200 and resp.json().get('page', 10000) == 10000:
            break
    
    return df

In [ ]:

# Use the function to retrieve movie list from TMDB via their API and create a dataframe
df = get_mtdb_movies()

In [ ]:
# Preview the dataframe
df.head()

In [ ]:
# Check the dimension of the dataframe
df.shape


In [ ]:
# Export initial df to csv since it is a lenghty process to import 500 requests
df.to_csv('raw_tmdb.csv')

In [ ]:
#Reset df index.  This will remove the duplicate index cause by the concaatenation of the df's
df2 = df.reset_index()

In [ ]:
# Preview df2
df2.head()

In [ ]:
# Remove irrelevent columns for the analysis (data cleaning process)
df2.drop(['index', 'video', 'poster_path', 'adult', 'backdrop_path', 'overview'], axis=1, inplace=True)

In [ ]:

# Checking df2
df2.head()

In [ ]:
# Output the column names to use in the duplicated subset parameter
df2.columns

In [ ]:
# Check for row duplication (data cleaning process).  Exclude column genre_ids becuase it contains list elements
df2.duplicated(subset=['popularity', 'vote_count', 'id',
                       'original_language', 'original_title',
                       'title', 'vote_average', 'release_date']).sum()

In [ ]:
# Check for duplicate title (data cleaning process)
df2.title.duplicated().sum()

In [ ]:
# Investigate the duplicate titles.  Preview of the duplicate titles
df2.loc[df2.title.duplicated(),:]

In [ ]:
# Spot checking the titles.  They appear to be re-make.  Will keep the records.
df2[(df2['title'] == 'Aladdin') | (df2['title'] == 'Rampage') | (df2['title'] == 'The Addams Family')]

In [ ]:
# Check for nan value (data cleaning process)
df2.isnull().sum()

In [ ]:
# Review the NaN in the release_date column as reported above
df2.loc[df2.release_date.isnull(), :]

In [ ]:
# 1) Since we are intertested in 'original_language' = en, we will drop zh and ta
# 2) lookk up the release_date for the other two movies online, index id 4873 & 6421

# Remove row 1333 & 9056
df2.drop(index=[1333, 9056], axis=0, inplace=True)

In [ ]:
# Check that zh & ta are in fact removed
df2.loc[df2.release_date.isnull(), :]

In [ ]:
#  Both 4873 and 6421 are 2020 films and has yet to anounce a release date set in 2020.  Remove from df.
df2.drop(index=[4873, 6421], axis=0, inplace=True)


In [ ]:

# Re check isnull status
df2.isnull().sum()

function get_movie_bud_rev explantions:

The TMDB API also provide a way to get more details about a movie, suh as the revenue and budget data, which are not included in the initial data gathering. They are crucial in our analysis.
We can slightly modify the function get_mtdb_movies to achieve this goal by changing the url parameter.

In [ ]:
# Create a function to send request to pull movie details.  Interested in the revenue and the budget values
def get_movie_bud_rev(movieid,rev_bud='revenue'):
    '''
    Init signature:
        get_movie_bud_rev(movie_id, rev_bud='revenue')
    Docstring:     
        get_movie_bud_rev(movie_id, rev_bud='revenue') -> dict
        Retrieve the revenue (default) or the budget info from the reponse object
    
    Parameters
    ----------
    movie_id: 
        TMDB movie id as int
    rev_bud: 
        A string input (revenue | budget) to represent the dictionary key to search in the response object

    '''
    # define get request parameters
    url = 'https://api.themoviedb.org/3/movie/' + str(movieid) + '?'
    url_param = {
                    'api_key': api_key,
                    'language': 'en-US',
                }
    resp = requests.get(url, params=url_param)
    if resp.status_code == 200:
        resp_bud_rev = resp.json()[rev_bud]
    else:
        resp_bud_rev = np.nan
  
        
    return resp_bud_rev

In [ ]:

# Create the revenue columns in the dataframe and populate the column using the get_movie_bud_rev function
df2['revenue'] = df2['id'].apply(lambda x: get_movie_bud_rev(x, 'revenue'))

In [ ]:
# Create the budget column in the dataframe and populate the column using the get_movie_bud_rev function
df2['budget'] = df2['id'].apply(lambda x: get_movie_bud_rev(x, 'budget'))

In [ ]:
# Preview revenue & budget column 
df2.head()

In [ ]:
# Check for NAN in revenue & budget coliumns
df2.revenue.isnull().sum(), df2.budget.isnull().sum()

In [ ]:
# Review the row to decide what to do
df2.loc[df2.revenue.isnull(), :]

In [ ]:
# Since the movie title is missing a lot of useful info, drop the row
df2.drop(index=[2599], axis=0, inplace=True)


Function get_genre explanation:

The genre classification are represented by interger ID. In order to translate to more meaning terms, we need to retrieve the corresponding name for those IDs.
API is available to pull this genre titles.
get_genre has the similar structure as get_movie_bud_rev function. We only need to modify the URL to extract the genre names.

In [ ]:

# Get the genre id classification from TMDB web site
def get_genre():
    '''
    Init signature:
        get_genre()
    Docstring:
        get_genre() -> dict
        Retrieve the genre IDs and names stored as key:value dictionary.
    Parameters:
        none
    
    '''
    # Create the response object
    g_resp = requests.get(
        'https://api.themoviedb.org/3/genre/movie/list?api_key='
        + str(api_key) + '&language=en-US'
        )
    # if get request is successful, store genres name in dict.
    if g_resp.status_code == 200:
        genre_dict =  sorted(g_resp.json()['genres'], key=lambda x: x['id'])
        
    return genre_dict

Function match_genre_id Explantions:

This function will retrieve the genre id list from the dataframe as the first step.
In a nested for loop, it retrieves the first element in the id to name mapping dictionary. Next it loop through the genre id list to find the matching key.
If key matched, it append the key value to the genre_name list object, which will be returned by the function to populate the new column in the dataframe.

In [ ]:
# map TMDB genre_id to genre_name
def match_genre_id(gid):
    '''
    Init signature:
        match_genre_id(gid)
    Docstring:
        match_genre_id(gid) -> list
        Take input genre id and match it to the "genre id:genre name" dict to retrieve the genre name
    Parameters:
        gid: The value in the genre_id column of the dataframe
    '''
    # Initiate an empty list to store the matching genre name
    genre_name = []
    # call get_genre function to pull the genre id/name from TMDB
    genre_dict = get_genre()
    # Match gerne id to pull gerne name
    for i in gid:
        for j in genre_dict:
            if j['id'] == i:
                genre_name.append(j['name'])
    
    return genre_name

In [ ]:

# Create a new coluimn to store genre_name
df2['genre_names'] = df2['genre_ids'].apply(lambda x: match_genre_id(x))

In [ ]:
# Took 20 min to run match_genre_id, create a backup
df2.to_csv('add_genre_name_tmdb.csv')

In [ ]:
# Convert column release_date from string to datatime data type
import datetime

df2['release_date'] = pd.to_datetime(df2['release_date'])

# Note to self: above method is more useful in comparison to using str yyyy-mm-dd to str yyyy
# df2['release_date'] = df2['release_date'].map(lambda x: x[0:4] if not pd.isnull(x) else x)

In [ ]:
# divide revenue column by 1,000,000 for easy readability and graphing
df2['revenue'] = round(df2['revenue'].div(1000000),2)

In [ ]:
# divide budget column by 1,000,000 for easy readability and graphing
df2['budget'] = round(df2['budget'].div(1000000),2)

In [ ]:
# Export the final dataframe for safeguard
df2.to_csv('final_tmdb.csv', index=False)

# More cleaning and merge the dataframes

In [ ]:
#read in the two csv files
df = pd.read_csv('movieData.csv')
df2 = pd.read_csv('final_tmdb.csv')

In [ ]:
#clean up the column titles a bit
df2.columns = map(str.title, df2.columns)
df2.rename(columns={'Genre_Names':'Genre'},inplace = True)


In [ ]:
#clean up the column titles and grab the year from release date so we an merge on title and year
df2['Year']=df2['Release_Date']
df2.dropna(subset = ['Year'], inplace = True)
df2['Year']=df2['Year'].apply(lambda x: x[0:4])
df2['Year']=df2['Year'].astype(int)

In [ ]:
#merge the dataframes
merged = pd.merge(left = df, right = df2, on=['Title', 'Year'])

In [ ]:
#drop some superfluous columns
merged.drop(['Id','Original_Language','Original_Title','Genre_Ids','Genre_y'],axis =1)

In [ ]:
#rename column
merged.rename(columns={'Genre_x':'Genre'},inplace = True)


In [ ]:
#clean up extra characters from stars columns
merged['Stars'] = merged['Stars'].apply(lambda x:x.strip('['))
merged['Stars'] = merged['Stars'].apply(lambda x:x.strip(']'))
merged['Stars'] = merged['Stars'].apply(lambda x:x.replace("'",''))
merged['Stars'] = merged['Stars'].apply(lambda x:x.replace("  ",' '))


In [ ]:
#export merged dataframe
merged.to_csv('mergedData.csv', index = False)